## Geospatial Hackathon Example
#### Populations effected by Hurricane Ida

This is an example of how you could use 3 of the tables provided to understand who is affected by hurricane IDA.  We will start by importing the basic libraries

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import pydeck as pdk 
import json


from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from snowflake.snowpark import Window


# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


Here is a dataset viewing all hurricane points from hurricane IDA

In [ ]:
select * from DATAOPS_EVENT_PROD.HACKATHON_DATASETS.HURRICANE_POINTS

Lets visualise the points using a heatmap

In [ ]:
tooltip = {
   "html": """<b>Name:</b> {NAME} <br> <b>USA Wind:</b> {USA_WIND} <br> <b>Hurricane Date:</b> {HURRICANE_DATE}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

hurricane_points = session.table('HACKATHON_DATASETS.HURRICANE_POINTS')
    

hurricane_pointspd = hurricane_points.to_pandas()
center = hurricane_points.agg(avg('LAT'),avg('LON'))

LAT = center.collect()[0][0]
LON = center.collect()[0][1]


h_points = pdk.Layer(
            'HeatmapLayer',
            data=hurricane_pointspd,
            get_position=['LON','LAT'],
            get_color='[41,181,232]',
            get_radius=10,
            pickable=True)

map = pdk.Deck(
    
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=800,
        
        ),

    layers= [h_points],tooltip=tooltip,
    map_style=None
    


)

st.pydeck_chart(map)

Take a look at the next dataset - these are the track lines of the hurricane.  Below, we are also visualising the linestrings.  The tracks are filtered on two states -  **Mississippi** and **Louisana**

In [ ]:
tooltip = {
   "html": """<b>Name:</b> {NAME} <br> <b>USA Wind:</b> {USA_WIND} <br> <b>Hurricane Date:</b> {HURRICANE_DATE}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

hurricane_tracks = session.table('HACKATHON_DATASETS.HURRICANE_TRACKS')
        #.with_column('LON',round('LON',2).astype(FloatType()))\
    #.with_column('LAT',round('LAT',2).astype(FloatType()))

hurricane_trackspd = hurricane_tracks.to_pandas()
hurricane_trackspd["GEO"] = hurricane_trackspd["GEO"].apply(lambda row: json.loads(row)["coordinates"])

st.write(hurricane_tracks)
center = hurricane_tracks.agg(avg('LAT'),avg('LON'))

LAT = center.collect()[0][0]
LON = center.collect()[0][1]


tracks_layer  = pdk.Layer(
        type="PathLayer",
        data=hurricane_trackspd,
        pickable=True,
        get_color=[170, 74, 68],
        width_scale=5,
        opacity = 1,
        width_min_pixels=5,
        get_path="GEO",
        get_width=200,
)

map = pdk.Deck(
    
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=800,
        
        ),

    layers= [tracks_layer],tooltip=tooltip,
    map_style=None
    


)

st.pydeck_chart(map)

You will now bring in the Population Census Blocks for **Louisiana** and **Mississippi**

In [ ]:
population = session.table('DATAOPS_EVENT_PROD.HACKATHON_DATASETS.POPULATION_CENSUS_BLOCK')
population

The census blocks are multi polygons - we will transform to polygons to use in **Pydeck**

In [ ]:
popf = population.join_table_function('flatten',
                                        call_function('ST_ASGEOJSON',
                                        col('GEO'))['coordinates']).drop('SEQ',
                                                                               'KEY',
                                                                               'PATH',
                                                                               'INDEX',
                                                                               'THIS')   
popf = popf.with_column('GEO',
                                to_geography(object_construct(lit('coordinates'),
                                                        to_array('VALUE'),
                                                        lit('type'),
                                                        lit('Polygon')))).drop('VALUE')
popf

Here we are going to join the population dataset with the hurricane tracks.  We will use the intersects to do this.  To avoid overlapping, we filter out duplicates due to one line intersecting multiple population blocks.

In [ ]:
data = popf.join(hurricane_tracks,call_function('ST_INTERSECTS',hurricane_tracks['GEO'],popf['GEO']),rsuffix='_hurricane_track')

### remove duplicates caused by ST_INTERSECTS - prioritise ones with largest housing units
window_spec = Window.partition_by(popf['OBJECTID']).order_by(col('P0050014').asc())
data = data.with_column('row_num', row_number().over(window_spec))#.qualify(col('row_num') == 1).drop('row_num')
data = data.filter(col('row_num') == 1).drop('row_num')

### simplify the polygons to reduce memory load in streamlit
data = data.with_column('GEO',call_function('ST_SIMPLIFY',col('GEO'),10))

### extract out coordinates to use with pydeck
data = data.with_column('COORDINATES',call_function('ST_ASGEOJSON',col('GEO'))['coordinates'])
data = data.cache_result()
data

Below is a map that shows all the polygons that were effected by the hurricane.

In [ ]:
import json
import streamlit as st
import pandas as pd
import pydeck as pdk
import json
import warnings
import altair as alt

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
session = get_active_session()
from snowflake.snowpark.functions import *
st.subheader('Affected Populations by the Hurricane')
### create a filter dropdown using distinct urban extent values

### create a center point - this time using the centroid method as we are visualising one polygon at a time
centre = data.with_column('CENTROID',call_function('ST_CENTROID',col('GEO')))
centre = centre.with_column('LON',call_function('ST_X',col('CENTROID')))
centre = centre.with_column('LAT',call_function('ST_Y',col('CENTROID')))

centrepd = centre.select('LON','LAT').to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]

### drop geo as we will be using coordinates field
data1 = data.drop('GEO')
data1 = data1.dropna()
datapd = data.to_pandas()

### minmax populationss for 
min_pop = datapd['P0030007'].min()
max_pop = datapd['P0030007'].max()

# Define the start and end colors for the gradient (RGBA format)
# Low population (min_pop) will be BLUE
COLOR_LOW_POP = [41, 181, 232, 255] # Blue (R, G, B, A)
# High population (max_pop) will be ORANGE
COLOR_HIGH_POP = [255, 159, 54, 255] # Orange (R, G, B, A)

def get_color_from_population(population):
    # Handle the edge case where all population values are the same
    if min_pop == max_pop:
        # If no variation, assign a mid-point color (or one of the extremes)
        return COLOR_HIGH_POP # Or COLOR_LOW_POP, or an average
    else:
        # Normalize population to a 0-1 range
        normalized_pop = (population - min_pop) / (max_pop - min_pop)

        # Interpolate between the start and end colors
        r = int(COLOR_LOW_POP[0] * (1 - normalized_pop) + COLOR_HIGH_POP[0] * normalized_pop)
        g = int(COLOR_LOW_POP[1] * (1 - normalized_pop) + COLOR_HIGH_POP[1] * normalized_pop)
        b = int(COLOR_LOW_POP[2] * (1 - normalized_pop) + COLOR_HIGH_POP[2] * normalized_pop)
        a = int(COLOR_LOW_POP[3] * (1 - normalized_pop) + COLOR_HIGH_POP[3] * normalized_pop) # Interpolate alpha too if needed, or keep fixed

        return [r, g, b, a]

datapd['fill_color'] = datapd['P0030007'].apply(get_color_from_population)

tooltip = {
   "html": """<b>Name:</b> {NAME} 
                <br> <b>USA Wind:</b> {USA_WIND} 
                <br> <b>Water Surfaces Sq Feet:</b> {AWATER}
                <br> <b>Populated Density:</b> {P001_CALC_PCTPOPDENSITY}
                <br> <b>Population County of Housing Units: </b> {P0020003}
                <br> <b>Population Count: </b> {P0030007}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}



# convert the dataframe to pandas and use a pandas lamda function to extract the coordinates out of each polygon.  
##pydeck only requires sets of coordinates in arrays, not the polygon itself


datapd["COORDINATES"] = datapd["COORDINATES"].apply(lambda row: json.loads(row))

col1, col2,col3,col4 = st.columns(4)
with col1:
    bar_data = data1.group_by('COUNTY')\
        .agg(avg('USA_WIND').alias('USA_WIND'))\
        .sort(col('USA_WIND').desc()).limit(5).to_pandas()
        
    c = alt.Chart(bar_data,height=500).mark_bar(color='#29B5E8')\
        .encode(x=alt.X("COUNTY", sort=None,title=None), y="USA_WIND")
    st.altair_chart(c, use_container_width=True)
with col2:
    bar_data = data1.group_by('COUNTY')\
        .agg(sum('P0030007').alias('POPULATION_COUNT'))\
        .sort(col('POPULATION_COUNT').desc()).limit(5).to_pandas()
        
    c = alt.Chart(bar_data,height=500).mark_bar(color='#29B5E8')\
        .encode(x=alt.X("COUNTY", sort=None,title=None), y="POPULATION_COUNT")
    st.altair_chart(c, use_container_width=True)

with col3:
    bar_data = data1.group_by('COUNTY')\
        .agg(sum('P0170022').alias('HOUSING_UNITS'))\
        .sort(col('HOUSING_UNITS').desc()).limit(5).to_pandas()
        
    c = alt.Chart(bar_data,height=500).mark_bar(color='#29B5E8')\
        .encode(x=alt.X("COUNTY", sort=None,title=None), y="HOUSING_UNITS")
    st.altair_chart(c, use_container_width=True)

with col4:
    bar_data = data1.group_by('COUNTY')\
        .agg(median('P0120047').alias('AGE'))\
        .sort(col('AGE').desc()).limit(5).to_pandas()
        
    c = alt.Chart(bar_data,height=500).mark_bar(color='#29B5E8')\
        .encode(x=alt.X("COUNTY", sort=None,title=None), y="AGE")
    st.altair_chart(c, use_container_width=True)



# Create data layer for each polygon
data_layer = pdk.Layer(
    "PolygonLayer",
    datapd,
    opacity=0.3,
    get_polygon="COORDINATES", 
    filled=True,
    get_fill_color="fill_color",
    get_line_color=[0, 0, 0],
    auto_highlight=True,
    pickable=True,
)

# Set the view on the map
view_state = pdk.ViewState(
    longitude=LON,
    latitude=LAT,
    zoom=6,  # Adjust zoom if needed
    pitch=0,
)



# Render the map with layer and tooltip
r = pdk.Deck(
    layers=[data_layer],
    initial_view_state=view_state,
    map_style=None,
    tooltip=tooltip)

st.pydeck_chart(r, use_container_width=True)



### Introducing a Filter

Here we are adding a filter by county - we will still render metrics and a map, but this time it is filtered by county and we are seeing the details by population block

In [ ]:
import json
import streamlit as st
import pandas as pd
import pydeck as pdk
import json
import warnings
import altair as alt

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
session = get_active_session()
from snowflake.snowpark.functions import *
st.subheader('Affected Populations by the Hurricane')
### create a filter dropdown using distinct urban extent values

countyfilter = data.select('COUNTY').distinct().sort('COUNTY')

choose_county = st.selectbox('Choose County:',countyfilter)

dataf = data.filter(col('COUNTY')==choose_county)



### create a center point - this time using the centroid method as we are visualising one polygon at a time
centre = dataf.with_column('CENTROID',call_function('ST_CENTROID',col('GEO')))
centre = centre.with_column('LON',call_function('ST_X',col('CENTROID')))
centre = centre.with_column('LAT',call_function('ST_Y',col('CENTROID')))

centrepd = centre.select('LON','LAT').to_pandas()
LON = centrepd.LON.iloc[0]
LAT = centrepd.LAT.iloc[0]

### drop geo as we will be using coordinates field
data1 = dataf.drop('GEO')
data1 = data1.dropna()
datapd = dataf.to_pandas()

### minmax populationss for 
min_pop = datapd['P0030007'].min()
max_pop = datapd['P0030007'].max()

# Define the start and end colors for the gradient (RGBA format)
# Low population (min_pop) will be BLUE
COLOR_LOW_POP = [41, 181, 232, 255] # Blue (R, G, B, A)
# High population (max_pop) will be ORANGE
COLOR_HIGH_POP = [255, 159, 54, 255] # Orange (R, G, B, A)

def get_color_from_population(population):
    # Handle the edge case where all population values are the same
    if min_pop == max_pop:
        # If no variation, assign a mid-point color (or one of the extremes)
        return COLOR_HIGH_POP # Or COLOR_LOW_POP, or an average
    else:
        # Normalize population to a 0-1 range
        normalized_pop = (population - min_pop) / (max_pop - min_pop)

        # Interpolate between the start and end colors
        r = int(COLOR_LOW_POP[0] * (1 - normalized_pop) + COLOR_HIGH_POP[0] * normalized_pop)
        g = int(COLOR_LOW_POP[1] * (1 - normalized_pop) + COLOR_HIGH_POP[1] * normalized_pop)
        b = int(COLOR_LOW_POP[2] * (1 - normalized_pop) + COLOR_HIGH_POP[2] * normalized_pop)
        a = int(COLOR_LOW_POP[3] * (1 - normalized_pop) + COLOR_HIGH_POP[3] * normalized_pop) # Interpolate alpha too if needed, or keep fixed

        return [r, g, b, a]

datapd['fill_color'] = datapd['P0030007'].apply(get_color_from_population)

tooltip = {
   "html": """<b>Name:</b> {NAME} 
                <br> <b>USA Wind:</b> {USA_WIND} 
                <br> <b>Water Surfaces Sq Feet:</b> {AWATER}
                <br> <b>Populated Density:</b> {P001_CALC_PCTPOPDENSITY}
                <br> <b>Population County of Housing Units: </b> {P0020003}
                <br> <b>Population Count: </b> {P0030007}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}



# convert the dataframe to pandas and use a pandas lamda function to extract the coordinates out of each polygon.  
##pydeck only requires sets of coordinates in arrays, not the polygon itself


datapd["COORDINATES"] = datapd["COORDINATES"].apply(lambda row: json.loads(row))

col1, col2,col3,col4 = st.columns(4)
with col1:
    bar_data = data1.group_by('NAME')\
        .agg(avg('USA_WIND').alias('USA_WIND'))\
        .sort(col('USA_WIND').desc()).limit(5).to_pandas()
        
    c = alt.Chart(bar_data,height=500).mark_bar(color='#29B5E8')\
        .encode(x=alt.X("NAME", sort=None,title=None), y="USA_WIND")
    st.altair_chart(c, use_container_width=True)
with col2:
    bar_data = data1.group_by('NAME')\
        .agg(sum('P0030007').alias('POPULATION_COUNT'))\
        .sort(col('POPULATION_COUNT').desc()).limit(5).to_pandas()
        
    c = alt.Chart(bar_data,height=500).mark_bar(color='#29B5E8')\
        .encode(x=alt.X("NAME", sort=None,title=None), y="POPULATION_COUNT")
    st.altair_chart(c, use_container_width=True)

with col3:
    bar_data = data1.group_by('NAME')\
        .agg(sum('P0170022').alias('HOUSING_UNITS'))\
        .sort(col('HOUSING_UNITS').desc()).limit(5).to_pandas()
        
    c = alt.Chart(bar_data,height=500).mark_bar(color='#29B5E8')\
        .encode(x=alt.X("NAME", sort=None,title=None), y="HOUSING_UNITS")
    st.altair_chart(c, use_container_width=True)

with col4:
    bar_data = data1.group_by('NAME')\
        .agg(median('P0120047').alias('AGE'))\
        .sort(col('AGE').desc()).limit(5).to_pandas()
        
    c = alt.Chart(bar_data,height=500).mark_bar(color='#29B5E8')\
        .encode(x=alt.X("NAME", sort=None,title=None), y="AGE")
    st.altair_chart(c, use_container_width=True)



# Create data layer for each polygon
data_layer = pdk.Layer(
    "PolygonLayer",
    datapd,
    opacity=0.3,
    get_polygon="COORDINATES", 
    filled=True,
    get_fill_color="fill_color",
    get_line_color=[0, 0, 0],
    auto_highlight=True,
    pickable=True,
)

# Set the view on the map
view_state = pdk.ViewState(
    longitude=LON,
    latitude=LAT,
    zoom=9,  # Adjust zoom if needed
    pitch=0,
)



# Render the map with layer and tooltip
r = pdk.Deck(
    layers=[data_layer],
    initial_view_state=view_state,
    map_style=None,
    tooltip=tooltip)

st.pydeck_chart(r, use_container_width=True)



## Add cortex to your visualisation

here we are using the data which we have filtered to prompt cortex to explain what the data is

In [ ]:
cortex_object = data1.drop('GEO_HURRICANE_TRACK','COORDINATES').select(array_agg(object_construct('*')).alias('OBJECT'))
cortex_object = cortex_object.with_column('prompt',lit('create a report in markdown based on the following dataset'))

insights = cortex_object.select(call_function('SNOWFLAKE.CORTEX.COMPLETE',
                                    'claude-4-sonnet',concat('prompt',
                                                            col('OBJECT').astype(StringType()))))

st.markdown(insights.collect()[0][0])


## USING H3

you could index the data into H3 hexagons - to do this, you would need to do the following:

- Convert each hurricane point to H3.  Here, I have chosen resolution 5
- use the coverage function in order to cover each Hurricane Track with the same H3 index resolution.
- use the coverage function in order to cover each population block with the same H3 index resolution.

In [ ]:
hurricane_pointsH3 = hurricane_points.with_column('H3',call_function('H3_POINT_TO_CELL_STRING',col('GEO'),5))

hurricane_pointsH3

In [ ]:
H3pop = population.with_column('H3',call_function('H3_TRY_COVERAGE_STRINGS',col('GEO'),5)).with_column('total_indexes',array_size('H3'))
H3pop = H3pop.join_table_function('flatten','H3').group_by('VALUE','STATE','COUNTY').agg(min('TOTAL_INDEXES').alias('ind_count'),avg('P0030007').alias('pop_count'))
H3pop = H3pop.select(col('VALUE').astype(StringType()).alias('H3'),'STATE','COUNTY',div0('pop_count','ind_count').alias('approx_population'))
H3pop

Next, we will load the H3 tracks and join with the H3 population blocks.

In [ ]:
hurricane_tracks = session.table('HACKATHON_DATASETS.HURRICANE_TRACKS')
H3_hurricane_tracks = hurricane_tracks.with_column('H3',call_function('H3_TRY_COVERAGE_STRINGS',col('GEO'),5))
H3_hurricane_tracks = H3_hurricane_tracks.join_table_function('flatten','H3').select(col('VALUE').astype(StringType()).alias('H3')).distinct()

H3_hurricane_tracks = H3_hurricane_tracks.join(H3pop,'H3')
H3_hurricane_tracks = H3_hurricane_tracks\
                .group_by('H3')\
                .agg(avg('APPROX_POPULATION').alias('APPROX_POPULATION'),
                     any_value('STATE').alias('STATE'),
                     any_value('COUNTY').alias('COUNTY'))
H3_hurricane_trackspd = H3_hurricane_tracks.to_pandas()
H3_hurricane_trackspd

Finally, you will visualise the results

In [ ]:
min_pop = H3_hurricane_trackspd['APPROX_POPULATION'].min()
max_pop = H3_hurricane_trackspd['APPROX_POPULATION'].max()

# Define the start and end colors for the gradient (RGBA format)
# Low population (min_pop) will be BLUE
COLOR_LOW_POP = [41, 181, 232, 255] # Blue (R, G, B, A)
# High population (max_pop) will be ORANGE
COLOR_HIGH_POP = [255, 159, 54, 255] # Orange (R, G, B, A)

def get_color_from_population(population):
    # Handle the edge case where all population values are the same
    if min_pop == max_pop:
        # If no variation, assign a mid-point color (or one of the extremes)
        return COLOR_HIGH_POP # Or COLOR_LOW_POP, or an average
    else:
        # Normalize population to a 0-1 range
        normalized_pop = (population - min_pop) / (max_pop - min_pop)

        # Interpolate between the start and end colors
        r = int(COLOR_LOW_POP[0] * (1 - normalized_pop) + COLOR_HIGH_POP[0] * normalized_pop)
        g = int(COLOR_LOW_POP[1] * (1 - normalized_pop) + COLOR_HIGH_POP[1] * normalized_pop)
        b = int(COLOR_LOW_POP[2] * (1 - normalized_pop) + COLOR_HIGH_POP[2] * normalized_pop)
        a = int(COLOR_LOW_POP[3] * (1 - normalized_pop) + COLOR_HIGH_POP[3] * normalized_pop) # Interpolate alpha too if needed, or keep fixed

        return [r, g, b, a]

H3_hurricane_trackspd['fill_color'] = H3_hurricane_trackspd['APPROX_POPULATION'].apply(get_color_from_population)

hurricane_pointsH3pd = hurricane_pointsH3.to_pandas()

h3points = pdk.Layer(
        "H3HexagonLayer",
        hurricane_pointsH3pd,
        pickable=False,
        stroked=True,
        filled=False,
        extruded=False,
        get_hexagon="H3",
        get_line_color=[0,0,0],
        line_width_min_pixels=2,
        opacity=0.4)

h3 = pdk.Layer(
        "H3HexagonLayer",
        H3_hurricane_trackspd,
        pickable=True,
        stroked=True,
        filled=True,
        extruded=True,
        get_hexagon="H3",
        get_fill_color="fill_color",
        line_width_min_pixels=0,
        opacity=0.4)

tooltip = {
   "html": """<b>H3:</b> {H3} <br> <b>Approx Population:</b> {APPROX_POPULATION}""",
   "style": {
       "width":"50%",
        "backgroundColor": "steelblue",
        "color": "white",
       "text-wrap": "balance"
   }
}

st.pydeck_chart(pdk.Deck(
    map_style=None,
    initial_view_state=pdk.ViewState(
        latitude=LAT,
        longitude=LON,
        zoom=5,
        height=600
        ),
    
layers= [h3,h3points], tooltip = tooltip

))